In [9]:
import time
import torch
from IDRiDClassifier import IDRiDClassifier
from IDRiDGradingDataset import IDRiDGradingDataset
import copy
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

In [11]:
# Prepare data

root_dir_train = 'B. Disease Grading/1. Original Images/a. Training Set/'
csv_file_train = 'B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv'

root_dir_test = 'B. Disease Grading/1. Original Images/a. Testing Set/'
csv_file_test = 'B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Testing Labels.csv'
bengraham = False

# transform (preprocessing)
scale = transforms.Resize((512, 512))
to_tensor = transforms.ToTensor()
composed = transforms.Compose([scale,
                               to_tensor])
# train and test set
train_set = IDRiDGradingDataset(csv_file_train, root_dir_train, composed, bengraham)
test_set = IDRiDGradingDataset(csv_file_test, root_dir_test, composed, bengraham)

#dataloaders
train_loader = DataLoader(train_set, batch_size=4,
                            shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=4,
                            shuffle=True, num_workers=0)


FileNotFoundError: [Errno 2] File b'B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Testing Labels.csv' does not exist: b'B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Testing Labels.csv'

In [12]:
# model
model = IDRiDClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

IDRiDClassifier(
  (Avg): AvgPool2d(kernel_size=4, stride=4, padding=0)
  (ReLU): ReLU()
  (Dropout): Dropout(p=0.1, inplace=False)
  (linear): Linear(in_features=2048, out_features=5, bias=True)
  (res101): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     

NameError: name 'model' is not defined